In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [2]:
#Gather data

boston_dataset = load_boston()
data = pd.DataFrame(data = boston_dataset.data, columns = boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis = 1)
features.head()

log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns = ['PRICE'])
target.shape

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (

(506, 1)

In [3]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

#property_stats = np.ndarray(shape = (1,11))
#property_stats[0][CRIME_IDX] = features['CRIM'].mean()
#property_stats[0][ZN_IDX] = features['ZN'].mean()
#property_stats[0][CHAS_IDX] = features['CHAS'].mean()


property_stats = features.mean().values.reshape(1,11)

In [24]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)

#Calculate the MSE and RMSE using sklearn

mse = mean_squared_error(target, fitted_vals)
rmse = mean_squared_error(target, fitted_vals, squared = False)
print(mse, rmse)

0.03516080084618688 0.18751213519713034


In [28]:
def get_log_estimate(nr_rooms,
                    students_per_classroom,
                    next_to_river = False,
                    high_confidence = True):
    #Configure property
    
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
    
    #Make prediction
    log_estimate = regr.predict(property_stats)
    
    #Calc range
    if high_confidence:
        #Do X
        upper_bound = log_estimate + 2*rmse
        lower_bound = log_estimate - 2*rmse
        interval = 95
    else:
        #Do y
        upper_bound = log_estimate + rmse
        lower_bound = log_estimate - rmse
        interval = 68
    
    return log_estimate[0][0], upper_bound[0][0], lower_bound[0][0], interval

In [29]:
get_log_estimate(3, 20, next_to_river = True, high_confidence = False)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


(2.776758191480399, 2.9642703266775294, 2.589246056283269, 68)

In [7]:
np.median(boston_dataset.target)

21.2

In [30]:
get_log_estimate(9, students_per_classroom = 15, next_to_river = False, high_confidence = False)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


(3.4027760019449502, 3.5902881371420805, 3.21526386674782, 68)

In [33]:
#Code that converts the log price estimate using 1970s prices (original prices) as well as the
#upper and the lower bound to today's prices. Values rounded to the nearest 1000 dollars.

ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)

log_estimate, upper_bound, lower_bound, interval = get_log_estimate(9, students_per_classroom = 15, next_to_river = False, high_confidence = False)

#Convert to today's dollars
dollar_est = np.e**log_estimate * 1000 * SCALE_FACTOR
dollar_hi = np.e**upper_bound * 1000 * SCALE_FACTOR
dollar_low = np.e**lower_bound * 1000 * SCALE_FACTOR


#Round the dollar values to nearest thousand
rounded_est = np.around(dollar_est, -3)
rounded_hi = np.around(dollar_hi, -3)
rounded_low = np.around(dollar_low, -3)

print(f'The estimated property value is {rounded_est}.')
print(f'At {interval}% confidence the valuation range is:')
print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

The estimated property value is 827000.0.
At 68% confidence the valuation range is:
USD 685000.0 at the lower end to USD 997000.0 at the high end.


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [49]:
def get_dollar_estimate(rm, ptratio, chas = False, large_range = True):
    
    """Estimate the price of a property in Boston.
    
    Keyword arguments:
    
    rm = Number of rooms
    ptratio = Number of students per classroom
    chas = if True the property is next to the river. Default is False
    large_range = True for a 95% prediction interval, False for a 68% interval.
    """
    
    log_estimate, upper_bound, lower_bound, interval = get_log_estimate(rm, ptratio, 
                                                                        next_to_river = chas, 
                                                                        high_confidence = large_range)
    
    if rm<1 or ptratio<1:
        print('That is unrealistic. Try again.')
        return
    
    
    #Convert to today's dollars
    dollar_est = np.e**log_estimate * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper_bound * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower_bound * 1000 * SCALE_FACTOR


    #Round the dollar values to nearest thousand
    rounded_est = np.around(dollar_est, -3)
    rounded_hi = np.around(dollar_hi, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f'The estimated property value is {rounded_est}.')
    print(f'At {interval}% confidence the valuation range is:')
    print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

In [48]:
get_dollar_estimate(rm = 4, ptratio = 10, chas = True)

The estimated property value is 704000.0.
At 95% confidence the valuation range is:
USD 484000.0 at the lower end to USD 1024000.0 at the high end.


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [50]:
import boston_valuation as val
val.get_dollar_estimate(6, 12, True)

0.03516080084618688 0.18751213519713034
The estimated property value is 783000.0.
At 95% confidence the valuation range is:
USD 538000.0 at the lower end to USD 1139000.0 at the high end.


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (